# TSIS 8 - PyGame

Это **#8** лабораторная работа по предмету *"Принципы Программирования 2".*

Студента: **Айбасов Даниал Елнарович**

**1. Racer**
Extra tasks to the given tutorial:

- Adding randomly appearing coins on the road
- Showing the number of collected coins in the top right corner
- Comment your code

**2. Snake**
Extend example project from lecture and add the following functionality:

- Checking for border (wall) collision and whether the snake is leaving the playing area
- Generate random position for food, so that it does not fall on a wall or a snake
- Add levels. For example, when the snake receives 3-4 foods or depending on score 
- Increase speed when the user passes to the next level
- Add counter to score and level
- Comment your code

**3. Paint**

- Draw rectangle
- Draw circle
- Eraser
- Color selection

In [1]:
import pygame
import sys
import random
import time
from pygame.locals import *

pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


---

# 1. Racer

In [12]:
# Initializing
pygame.init()

# Setting up FPS
FPS = 60
FramePerSec = pygame.time.Clock()

# Defining colors
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Screen dimensions and other variables
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 8
SCORE = 0
COIN_SCORE = 0

# Setting up fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over_text = font.render("Game Over", True, BLACK)

# Loading assets, background
background = pygame.image.load("assets/AnimatedStreet.png")

# Creating display surface
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Game")

# Coin class
class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/coin.png")
        self.image = pygame.transform.scale(self.image, (30, 30))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global COIN_SCORE
        self.rect.move_ip(0, 10)
        if self.rect.top > SCREEN_HEIGHT:
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

# Enemy class
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

# Player class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("assets/Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if pressed_keys[K_LEFT] and self.rect.left > 0:
            self.rect.move_ip(-7, 0)
        if pressed_keys[K_RIGHT] and self.rect.right < SCREEN_WIDTH:
            self.rect.move_ip(7, 0)

# Setting up sprites
P1 = Player()
E1 = Enemy()
coin = Coin()

# Creating sprite groups
enemies = pygame.sprite.Group()
enemies.add(E1)
all_sprites = pygame.sprite.Group(P1, E1, coin)
coin_group = pygame.sprite.Group(coin)

# Adding a new event for increasing speed
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

# Game Loop
while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.5
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # Background and scores
    DISPLAYSURF.blit(background, (0, 0))
    score_text = font_small.render(f'Score: {SCORE}', True, BLACK)
    coin_text = font_small.render(f'Coins: {COIN_SCORE}', True, BLACK)
    DISPLAYSURF.blit(score_text, (10, 10))
    DISPLAYSURF.blit(coin_text, (300, 10))

    # Move and redraw all sprites
    for entity in all_sprites:
        DISPLAYSURF.blit(entity.image, entity.rect)
        entity.move()

    # Coin collection
    if pygame.sprite.spritecollideany(P1, coin_group):
        COIN_SCORE += 1
        for entity in coin_group:
            entity.rect.top = 0
            entity.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    # Collision with enemy
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.Sound('assets/crash.wav').play()
        time.sleep(0.5)

        # Game over screen
        DISPLAYSURF.fill(RED)
        DISPLAYSURF.blit(game_over_text, (30, 250))
        pygame.display.update()
        

    pygame.display.update()
    FramePerSec.tick(FPS)


SystemExit: 

---

# 2. Snake

In [14]:
# Initialize pygame
pygame.init()

# Screen and game settings
WIDTH, HEIGHT = 800, 800
SCREEN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")

# Colors
RED = (255, 0, 0)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
MAGENTA = (255, 0, 255)
WHITE = (255, 255, 255)

# Game settings
BLOCK_SIZE = 40
SPEED = 5
SCORE = 0
level = 1
clock = pygame.time.Clock()

# Fonts
font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over_text = font.render("Game Over", True, BLACK)
win_text = font.render("Congratulations", True, BLACK)

# Game classes
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Snake:
    def __init__(self):
        self.body = [
            Point(x=WIDTH // BLOCK_SIZE // 2, y=HEIGHT // BLOCK_SIZE // 2),
            Point(x=WIDTH // BLOCK_SIZE // 2 + 1, y=HEIGHT // BLOCK_SIZE // 2),
        ]

    def draw(self):
        head = self.body[0]
        pygame.draw.rect(SCREEN, RED, pygame.Rect(head.x * BLOCK_SIZE, head.y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))
        for part in self.body[1:]:
            pygame.draw.rect(SCREEN, BLUE, pygame.Rect(part.x * BLOCK_SIZE, part.y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))

    def move(self, dx, dy):
        for idx in range(len(self.body) - 1, 0, -1):
            self.body[idx].x = self.body[idx - 1].x
            self.body[idx].y = self.body[idx - 1].y
        self.body[0].x += dx
        self.body[0].y += dy

        # Wrap around screen edges
        self.body[0].x %= WIDTH // BLOCK_SIZE
        self.body[0].y %= HEIGHT // BLOCK_SIZE

    def check_collision(self, food):
        return self.body[0].x == food.location.x and self.body[0].y == food.location.y

    def check_collision_wall(self, wall):
        for w in wall.walls:
            if w.x == self.body[0].x and w.y == self.body[0].y:
                return True
        return False

class Food:
    def __init__(self, x, y):
        self.location = Point(x, y)

    def draw(self):
        pygame.draw.rect(SCREEN, GREEN, pygame.Rect(self.location.x * BLOCK_SIZE, self.location.y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))

class Wall:
    def __init__(self, x_list, y_list):
        self.walls = [Point(x, y) for x, y in zip(x_list, y_list)]

    def draw(self):
        for wall in self.walls:
            pygame.draw.rect(SCREEN, MAGENTA, pygame.Rect(wall.x * BLOCK_SIZE, wall.y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))

# Draw grid
def draw_grid():
    for x in range(0, WIDTH, BLOCK_SIZE):
        pygame.draw.line(SCREEN, WHITE, (x, 0), (x, HEIGHT), 1)
    for y in range(0, HEIGHT, BLOCK_SIZE):
        pygame.draw.line(SCREEN, WHITE, (0, y), (WIDTH, y), 1)

# Main game loop
def main():
    global SCORE, SPEED, level
    running = True
    snake = Snake()
    food = Food(5, 5)

    # Levels
    level1 = Wall(x_list=range(20), y_list=[0]*20)
    level2 = Wall(x_list=[1]*20, y_list=range(20))
    level3 = Wall(x_list=[18]*20, y_list=range(20))
    current_level = level1

    dx, dy = 0, 0

    while running:
        SCREEN.fill(BLACK)
        
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # Movement controls
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    dx, dy = 0, -1
                elif event.key == pygame.K_DOWN:
                    dx, dy = 0, 1
                elif event.key == pygame.K_RIGHT:
                    dx, dy = 1, 0
                elif event.key == pygame.K_LEFT:
                    dx, dy = -1, 0

        # Display score and level
        score_text = font_small.render(f'Score: {SCORE}', True, RED)
        level_text = font_small.render(f'Level: {level}', True, RED)
        SCREEN.blit(score_text, (10, 10))
        SCREEN.blit(level_text, (10, 50))

        # Level management
        if SCORE == 0:
            current_level = level1
        elif SCORE == 4:
            current_level = level2
            SPEED = 8
            level = 2
        elif SCORE == 8:
            current_level = level3
            SPEED = 12
            level = 3
        elif SCORE == 12:
            SCREEN.fill(RED)
            SCREEN.blit(win_text, (7 * BLOCK_SIZE, 10 * BLOCK_SIZE))
            pygame.display.update()
            time.sleep(2)
            pygame.quit()
            return

        current_level.draw()
        snake.move(dx, dy)

        # Food collision
        if snake.check_collision(food):
            SCORE += 1
            snake.body.append(Point(snake.body[-1].x, snake.body[-1].y))
            food.location.x = random.randint(0, WIDTH // BLOCK_SIZE - 1)
            food.location.y = random.randint(0, HEIGHT // BLOCK_SIZE - 1)

        # Wall collision
        if snake.check_collision_wall(current_level):
            SCREEN.fill(RED)
            SCREEN.blit(game_over_text, (7 * BLOCK_SIZE, 10 * BLOCK_SIZE))
            pygame.display.update()
            time.sleep(2)
            pygame.quit()
            return

        # Draw everything
        snake.draw()
        food.draw()
        draw_grid()
        pygame.display.flip()
        clock.tick(SPEED)

# Run the game
if __name__ == '__main__':
    main()


---

# 3. Paint

In [3]:
def main():
    pygame.init()
    screen = pygame.display.set_mode((640, 480))
    clock = pygame.time.Clock()
    
    radius = 15
    mode = 'blue'
    painting_mode = 'circle'
    points = []
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                running = False
                
        pressed = pygame.key.get_pressed()
        alt_held = pressed[pygame.K_LALT] or pressed[pygame.K_RALT]
        ctrl_held = pressed[pygame.K_LCTRL] or pressed[pygame.K_RCTRL]
        
        for event in pygame.event.get():
            # Determine if X was clicked, or Ctrl+W or Alt+F4 was used
            if event.type == pygame.QUIT:
                return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w and ctrl_held:
                    return
                if event.key == pygame.K_F4 and alt_held:
                    return
                if event.key == pygame.K_ESCAPE:
                    return
                
                # Determine if a letter key was pressed for color or mode change
                if event.key == pygame.K_r:
                    mode = 'red'
                elif event.key == pygame.K_g:
                    mode = 'green'
                elif event.key == pygame.K_b:
                    mode = 'blue'
                elif event.key == pygame.K_l:
                    painting_mode = 'rect'
                elif event.key == pygame.K_c:
                    painting_mode = 'circle'
                elif event.key == pygame.K_e:
                    painting_mode = 'eraser'
            
            # Handle mouse events for radius change
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # Left click grows radius
                    radius = min(200, radius + 1)
                elif event.button == 3:  # Right click shrinks radius
                    radius = max(1, radius - 1)
            
            # Track mouse position to draw lines
            if event.type == pygame.MOUSEMOTION:
                position = event.pos
                points.append(position)
                points = points[-256:]
        
        # Clear the screen for redrawing
        screen.fill((0, 0, 0))
        
        # Draw all points
        for i in range(len(points) - 1):
            drawLineBetween(screen, i, points[i], points[i + 1], radius, mode, painting_mode)
        
        pygame.display.flip()
        clock.tick(60)

def drawLineBetween(screen, index, start, end, width, color_mode, painting_mode):
    # Define color gradients based on index for a smooth gradient
    c1 = max(0, min(255, 2 * index - 256))
    c2 = max(0, min(255, 2 * index))
    
    if color_mode == 'blue':
        color = (c1, c1, c2)
    elif color_mode == 'red':
        color = (c2, c1, c1)
    elif color_mode == 'green':
        color = (c1, c2, c1)
    
    dx = start[0] - end[0]
    dy = start[1] - end[1]
    iterations = max(abs(dx), abs(dy))
    
    for i in range(iterations):
        progress = i / iterations
        aprogress = 1 - progress
        x = int(aprogress * start[0] + progress * end[0])
        y = int(aprogress * start[1] + progress * end[1])
        
        if painting_mode == 'circle':
            pygame.draw.circle(screen, color, (x, y), width)
        elif painting_mode == 'rect':
            pygame.draw.rect(screen, color, (x, y, width + 30, width + 30))
        elif painting_mode == 'eraser':
            pygame.draw.rect(screen, (0, 0, 0), (x, y, width, width))

# Run the program
if __name__ == '__main__':
    main()


SystemExit: 

C:\Users\ASUS\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import pygame

def main():
    pygame.init()
    screen = pygame.display.set_mode((640, 480))
    clock = pygame.time.Clock()
    
    radius = 15
    x = 0
    y = 0
    mode = 'blue'
    points = []
    painting_mode = 'circle'
    
    while True:
        pressed = pygame.key.get_pressed()
        alt_held = pressed[pygame.K_LALT] or pressed[pygame.K_RALT]
        ctrl_held = pressed[pygame.K_LCTRL] or pressed[pygame.K_RCTRL]
        
        for event in pygame.event.get():
            # determine if X was clicked, or Ctrl+W or Alt+F4 was used
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                running = False
                #return
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w and ctrl_held:
                    return
                if event.key == pygame.K_F4 and alt_held:
                    return
                if event.key == pygame.K_ESCAPE:
                    return

                # determine if a letter key was pressed
                if event.key == pygame.K_r:
                    mode = 'red'
                elif event.key == pygame.K_g:
                    mode = 'green'
                elif event.key == pygame.K_b:
                    mode = 'blue'
                elif event.key == pygame.K_l:
                    painting_mode = 'rect'
                elif event.key == pygame.K_c:
                    painting_mode = 'circle'
                elif event.key == pygame.K_e:
                    painting_mode = 'eraser'

            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:  # left click grows radius
                    radius = min(200, radius + 1)
                elif event.button == 3:  # right click shrinks radius
                    radius = max(1, radius - 1)

            if event.type == pygame.MOUSEMOTION:
                # if mouse moved, add point to list
                position = event.pos
                points = points + [position]
                points = points[-256:]
        
        screen.fill((0, 0, 0))

        # draw all points
        i = 0
        while i < len(points) - 1:
            drawLineBetween(screen, i, points[i], points[i + 1], radius, mode, painting_mode)
            i += 1

        pygame.display.flip()
        clock.tick(60)

def drawLineBetween(screen, index, start, end, width, color_mode, painting_mode):
    c1 = max(0, min(255, 2 * index - 256))
    c2 = max(0, min(255, 2 * index))
    
    if color_mode == 'blue':
        color = (c1, c1, c2)
    elif color_mode == 'red':
        color = (c2, c1, c1)
    elif color_mode == 'green':
        color = (c1, c2, c1)
    
    dx = start[0] - end[0]
    dy = start[1] - end[1]
    iterations = max(abs(dx), abs(dy))
    
    for i in range(iterations):
        progress = 1.0 * i / iterations
        aprogress = 1 - progress
        x = int(aprogress * start[0] + progress * end[0])
        y = int(aprogress * start[1] + progress * end[1])
        if painting_mode == 'circle':
            pygame.draw.circle(screen, color, (x, y), width)
        elif painting_mode == 'rect':
            pygame.draw.rect(screen, color, (x, y, width + 30, width + 30))
        elif painting_mode == 'eraser':
            pygame.draw.rect(screen, (0, 0, 0), (x, y, width, width))

main()


SystemExit: 

C:\Users\ASUS\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
